In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim.models import word2vec
import jieba
import os
from collections import Counter
import numpy as np

In [ ]:
# 处理语料：word2vec中需要sens是经过预处理且用空格进行分词的
def readTxt(txtpath):
  # txtpath="/content/drive/MyDrive/LSTM/dataG"
  txtfiles=os.listdir(txtpath)
  txtsList=[]
  for file in txtfiles:
    position = txtpath + '/' + file
    with open(position,"r",encoding='UTF-8') as f:   # 打开文件
      lines = f.readlines()
      for line in lines:
        if line!='\n':
          txtsList.append(line)  # 文本文件中每行为列表中一个元素
  text = ""
  for line in txtsList:
    for tok in jieba.lcut(line):
      text += tok
      text += ' '
  return text # 返回用空格分割的词语的文本字符串

In [ ]:
# 获取双向索引词典
# {词：索引}字典
def get_dict_tokenToIndex(data):
  list_dict=Counter(data).most_common()
  dict_tokenToIndex={}
  i=0
  for tup in list_dict:
    key , value = tup
    dict_tokenToIndex[key]=i
    i+=1
  # print(dict_token)
  return dict_tokenToIndex

# {索引：词}字典
def get_dict_indexTotoken(dict_tokenToIndex):
  dict_indexTotoken={}
  for key , value in dict_tokenToIndex.items():
    dict_indexTotoken[value] = key
  return dict_indexTotoken

In [ ]:
def processed1(txtpath,cutpath,tokToInt_path,intToTok_path,wordsnum_path):
  texts=readTxt(txtpath)
  with open(cutpath,'w') as f: # 存储用空格分割的词语的文本字符串到txt文件，便于之后word2vec.LineSentence的读取
    f.write(texts) 
  texts=texts.split(' ')
  # 获取词典
  dict_tokToInt = get_dict_tokenToIndex(texts)
  dict_intToTok = get_dict_indexTotoken(dict_tokToInt)
  # 存储
  np.save(tokToInt_path, dict_tokToInt)
  np.save(intToTok_path, dict_intToTok)
  np.save(wordsnum_path, len(dict_tokToInt))

In [ ]:
def train_word2vec(cutpath,wvsavepath):
  # 读取分词后的文本文件
  sentences=word2vec.LineSentence(cutpath)
  # 训练模型，词向量的长度设置为128，迭代次数为10，0即CBOW模型,1采用skip-gram模型，模型保存为bin格式
  model = gensim.models.Word2Vec(sentences, size=128, window=5 , min_count=1 , sg=0, iter=10)  
  model.wv.save_word2vec_format(wvsavepath, binary=True)

In [ ]:
def getWordEmbedding(intToTok_path,wvsavepath,wordsweigth_path):
  dict_intToTok = np.load(intToTok_path,allow_pickle=True).item()
  wordVec = gensim.models.KeyedVectors.load_word2vec_format(wvsavepath, binary=True)
  vocab = []
  wordEmbedding = []
  for index , word in dict_intToTok.items():
    try:
      vector = wordVec.wv[word]
      vocab.append(index)
      wordEmbedding.append(vector)
    except:
      print(str(index) + ":" + word + "不存在于词向量中")
      wordEmbedding.append([0]*128)
      vocab.append(index)
  print("词语数共：",len(vocab))
  np.save(wordsweigth_path,np.array(wordEmbedding))    # 词向量表示为num*dim 
  return np.array(wordEmbedding)

In [ ]:
processed1(txtpath,cutpath,tokToInt_path,intToTok_path,wordsnum_path)
train_word2vec(cutpath,wvsavepath)

In [ ]:
wordEmbedding = getWordEmbedding(intToTok_path,wvsavepath,wordsweigth_path)

In [ ]:
wordEmbedding